In [1]:
# Imports and configuration
%load_ext autoreload
%autoreload 2

import pandas as pd
import src.utils.file_utils as fu

In [2]:
# Read in raw data file
typed_hmda_data = fu.load_parquet("hmda_2024_typed")
print(typed_hmda_data.shape)

Loading dataset from /Users/c1burns/Documents/UTD/BUAN 6341/project_repo/data/interim/hmda_2024_typed.parquet
(8841112, 76)


In [5]:
# Verify target variable is binary
typed_hmda_data["approved_flag"].dtype

bool[pyarrow]

In [6]:
# Verify target variable values are all populated True/False
typed_hmda_data["approved_flag"].value_counts()

approved_flag
True     6693164
False    2147948
Name: count, dtype: int64[pyarrow]